In [2]:
import re
import string

def improved_gujarati_preprocessing(input_file_path, output_file_path):
    """
    Improved preprocessing pipeline for Gujarati OCR text from NCERT textbooks
    """
    
    print(f"Reading from: {input_file_path}")
    with open(input_file_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    # --- Stage 1: Basic Cleaning ---
    print("Performing basic cleaning...")
    
    # Remove extra whitespace but preserve sentence structure
    text = re.sub(r'\s+', ' ', raw_text)
    
    # Remove page numbers and isolated digits but keep numbers in context
    text = re.sub(r'\b\d+\b', '', text)
    
    # Remove special characters but preserve Gujarati punctuation
    gujarati_punctuation = '।॥॰'
    all_punctuation = string.punctuation + gujarati_punctuation
    text = re.sub(f'[^{all_punctuation}\\u0A80-\\u0AFF\\s]', '', text)
    
    # --- Stage 2: Content Extraction (More Flexible) ---
    print("Extracting relevant content...")
    
    # Look for chapter/content indicators
    chapter_indicators = [
        'પ્રકરણ', 'પ્રકરણો', 'જીવવિજ્ઞાન', 'જૈવ', 'સજીવ', 
        'વનસ્પતિ', 'પ્રાણી', 'વર્ગીકરણ', 'સૃષ્ટિ'
    ]
    
    # Find the main content by looking for these indicators
    lines = text.split('.')
    content_lines = []
    in_content = False
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Check if this line contains content indicators
        gujarati_chars = sum(1 for char in line if '\u0A80' <= char <= '\u0AFF')
        if gujarati_chars > 10:  # Reasonable Gujarati content
            if any(indicator in line for indicator in chapter_indicators) or in_content:
                in_content = True
                content_lines.append(line)
    
    if not content_lines:
        print("No specific content markers found. Using entire text.")
        content_lines = [line for line in lines if len(line.strip()) > 20]
    
    # --- Stage 3: Sentence Reconstruction ---
    print("Reconstructing sentences...")
    
    # Reconstruct proper sentences
    reconstructed_text = ' '.join(content_lines)
    
    # Better sentence splitting using Gujarati punctuation
    sentences = []
    current_sentence = []
    
    for char in reconstructed_text:
        current_sentence.append(char)
        if char in '।.':
            sentence = ''.join(current_sentence).strip()
            if len(sentence) > 10:  # Minimum sentence length
                sentences.append(sentence)
            current_sentence = []
    
    # Add any remaining text
    if current_sentence:
        final_sentence = ''.join(current_sentence).strip()
        if len(final_sentence) > 10:
            sentences.append(final_sentence)

    # --- Stage 4: Advanced Cleaning ---
    print("Performing advanced cleaning...")
    
    # Gujarati stop words (expanded list)
    gujarati_stop_words = {
        'અને', 'આ', 'એ', 'છે', 'માં', 'થી', 'તો', 'જ', 'ને', 'માટે', 
        'હોય', 'પણ', 'તે', 'કે', 'ના', 'નો', 'ની', 'નું', 'એક', 'પર',
        'કરે', 'છો', 'કરવા', 'છાત્ર', 'છાત્રા', 'છીએ', 'અહીં', 'જાઓ',
        'હવે', 'બનાવે', 'બધા', 'બહુ', 'આપે', 'દ્વારા', 'હું', 'તમે'
    }
    
    cleaned_sentences = []
    
    for sentence in sentences:
        # Tokenize while preserving words with mixed content
        words = sentence.split()
        cleaned_words = []
        
        for word in words:
            # Keep words that have substantial Gujarati content
            gujarati_chars = sum(1 for char in word if '\u0A80' <= char <= '\u0AFF')
            total_chars = len(word)
            
            if total_chars > 0 and (gujarati_chars / total_chars) > 0.6:
                # Remove stop words
                if word not in gujarati_stop_words:
                    cleaned_words.append(word)
        
        if len(cleaned_words) >= 3:  # Minimum 3 words per sentence
            cleaned_sentence = ' '.join(cleaned_words)
            cleaned_sentences.append(cleaned_sentence)

    # --- Stage 5: Save Processed Data ---
    print(f"Saving processed data to: {output_file_path}")
    
    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        for i, sentence in enumerate(cleaned_sentences, 1):
            f_out.write(f"{sentence}\n")
    
    print("-" * 50)
    print(f"Processing complete!")
    print(f"Original sentences: {len(sentences)}")
    print(f"Cleaned sentences: {len(cleaned_sentences)}")
    print(f"Successfully saved to '{output_file_path}'")
    print("-" * 50)
    
    return cleaned_sentences

# Run the improved preprocessing
import os
current_dir = os.getcwd()
input_file = os.path.join(current_dir, '../../data/processed/class12_maths.txt')
output_file = os.path.join(current_dir, '../../data/next/class12_maths_prepro.txt')

processed_sentences = improved_gujarati_preprocessing(input_file, output_file)

Reading from: c:\Users\Mitansh Kanani\Desktop\college\sem 7\NLP\NLP_LAB_GYANGUJ\src\preprocessing\../../data/processed/class12_maths.txt
Performing basic cleaning...
Extracting relevant content...
Reconstructing sentences...
Performing advanced cleaning...
Saving processed data to: c:\Users\Mitansh Kanani\Desktop\college\sem 7\NLP\NLP_LAB_GYANGUJ\src\preprocessing\../../data/next/class12_maths_prepro.txt
--------------------------------------------------
Processing complete!
Original sentences: 54
Cleaned sentences: 50
Successfully saved to 'c:\Users\Mitansh Kanani\Desktop\college\sem 7\NLP\NLP_LAB_GYANGUJ\src\preprocessing\../../data/next/class12_maths_prepro.txt'
--------------------------------------------------


In [3]:
def quality_check(output_file_path, sample_size=10):
    """
    Check the quality of processed data
    """
    print("Performing quality check...")
    
    with open(output_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"Total sentences: {len(lines)}")
    print("\nSample of processed sentences:")
    print("-" * 50)
    
    for i in range(min(sample_size, len(lines))):
        print(f"{i+1}. {lines[i].strip()}")
        print()

# Check the quality
quality_check(output_file)

Performing quality check...
Total sentences: 50

Sample of processed sentences:
--------------------------------------------------
1. પ્રકરણ પ્રકરણ પ્રકરણ પ્રકરણ અનુક્રમણિકા સંબંધ વિધેય (રહાશ ડોળ ત્રિકોણમિતીય પ્રતિવિધેયો પ્રાસ્તાવિક સંબંધોના પ્રકાર વિધેયોના પ્રકાર વિધેયોનું સંયોજન વ્યસ્તસંપન્ન વિધેય પ્રાસ્તાવિક પાયાની સંકલ્પના ત્રિકોણમિતીય પ્રતિવિધેયોના ગુણધર્મો શ્રેણિક પ્રાસ્તાવિક શ્રેણિક શ્રેણિકના પ્રકારો શ્રેણિક પરની પ્રક્રિયાઓ પરિવર્ત શ્રેણિક સંમિત વિસંમિત શ્રેણિક વ્યસ્તસંપન્ન શ્રેણિક (સામાન્ય શ્રેણિક) પ્રાસ્તાવિક નિશ્ચાયક ત્રિકોણનું ક્ષેત્રફળ ઉપનિશ્ચાયક સડઅવયવ સહઅવયવજ વસ્ત શ્રેણિક નિશ્ચાયક શ્રેણિકના ઉપયોગો ડા ડા ઊં ઊં પ્રકરણ સાતત્ય વિકલનીયતા રલ લા-લાઈંશાળા॥)/) પ્રાસ્તાવિક ઘાતાંકીય લઘુગણકીય વિધેયો લઘુગણકીય વિકલન પ્રચલ વિધેયનું વિકલિત હિતીય કક્ષાનો વિકલિત પ્રકરણ વિકલિતના ઉપયોગો &)[પાંલ્ળં લાંપાં) પ્રાસ્તાવિક રાશિમાં થતા ફેરફારનો દર વધતાં તથા ઘટતાં વિધેયો મહત્તમ તથા ન્યૂનતમ મૂલ્યો પરિશિષ્ટ ગણિતમાં સાબિતીઓ પ્રાસ્તાવિક સાબિતી શું પરિશિષ્ટ ગાણિતિક મૉડેલિંગ (પશ૦૦ પ્રાસ્તાવિક ગાણિતિક મોડે